# Primary School Rankings

There is no official primary school ranking done by the Ministry of Education. However, we adopt the "wisdom of the crowd" and use application details in 2021 to determine a prmary schools popularity.

Popularity is calculated as total applications recieved over all phases divided by the number of vacancies.

In [1]:
# import libs

import requests
import pandas as pd
import re
from MyCreds.mycreds import Capstone_AWS_PG
from sqlalchemy import create_engine

# grab application table from website
url = 'https://sgschooling.com/year/2021/all'

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)

# load into dataframe
primary_ranks = pd.read_html(r.text)[0]
primary_ranks

,School,Phase 1,2A(1),2A(2),2B,2C,2C(S),3
0,Admiralty,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,↳ Vacancy (210),170.0,87.0,87.0,50,50,0,0
2,↳ Applied,83.0,0.0,28.0,60,93,0,-
3,↳ Taken,83.0,0.0,28.0,50SC1-2,50SC<1,0,-
4,Ahmad Ibrahim,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
719,↳ Taken,65.0,0.0,71.0,0,144SC<1#,0,-
720,Northshore,NaN,NaN,NaN,NaN,NaN,NaN,NaN
721,↳ Vacancy (240),200.0,194.0,194.0,117,233,140,61
722,↳ Applied,6.0,0.0,0.0,1,93,79,-


## Clean the data and calculate popularity

In [2]:
# clean and process data

# set up some empty lists
schools = []
vacancy = []
applied = []

# iterate through each row of the dataframe
for i in primary_ranks.iterrows():
    # if it doesn't start with a ↳ it is the name of the school
    if not i[1][0].startswith("↳"):
        schools.append(i[1][0])
    # vacancies for each school is stored in within the (xxx)
    elif i[1][0].startswith("↳ Vacancy"):
        vacancy.append(re.findall(r'\d+',i[1][0])[0])
    # applications for each phase are in Applied
    elif i[1][0].startswith("↳ Applied"):
        applied_per_phase = []
        # if we are in the applied row
        for idx,num in enumerate(i[1]):
            try:
                # add the numbers to a list except the first
                applied_per_phase.append(int(num))
            except ValueError:
                # error when num is a '-' and not zero/or convertable to int like at idx 0
                pass
        applied.append(sum(applied_per_phase))

# put data into a data frame
primary_ranks_cleaned = pd.DataFrame(
    {
        'school_name' : schools,
        'vacancies' : vacancy,
        'total_application' : applied
    }
)

# add popularity
primary_ranks_cleaned['popularity'] = primary_ranks_cleaned['total_application'].astype(int)/primary_ranks_cleaned['vacancies'].astype(int)

# clean school name to match strings
primary_ranks_cleaned['match_on'] = primary_ranks_cleaned['school_name'].str.upper()
primary_ranks_cleaned['match_on'] = primary_ranks_cleaned['match_on'].str.replace('PRIMARY','')
primary_ranks_cleaned['match_on'] = primary_ranks_cleaned['match_on'].str.replace('SCHOOL','')
primary_ranks_cleaned['match_on'] = primary_ranks_cleaned['match_on'].str.replace('’','')
primary_ranks_cleaned['match_on'] = primary_ranks_cleaned['match_on'].str.replace('\'','')
primary_ranks_cleaned['match_on'] = primary_ranks_cleaned['match_on'].str.replace('.','')
primary_ranks_cleaned['match_on'] = primary_ranks_cleaned['match_on'].str.replace(' ','')
primary_ranks_cleaned['match_on'] = primary_ranks_cleaned['match_on'].str.replace('(','')
primary_ranks_cleaned['match_on'] = primary_ranks_cleaned['match_on'].str.replace(')','')
primary_ranks_cleaned['match_on'] = primary_ranks_cleaned['match_on'].str.strip()


primary_ranks_cleaned

/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:47: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:49: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


,school_name,vacancies,total_application,popularity,match_on
0,Admiralty,210,264,1.257143,ADMIRALTY
1,Ahmad Ibrahim,210,84,0.400000,AHMADIBRAHIM
2,Ai Tong,300,364,1.213333,AITONG
3,Alexandra,200,223,1.115000,ALEXANDRA
4,Anchor Green,240,271,1.129167,ANCHORGREEN
...,...,...,...,...,...
176,Zhangde,210,223,1.061905,ZHANGDE
177,Zhenghua,240,253,1.054167,ZHENGHUA
178,Zhonghua,180,83,0.461111,ZHONGHUA
179,Valour,280,295,1.053571,VALOUR


## Sanity Checking

In [3]:
primary_ranks_cleaned.sort_values(by=['popularity'],ascending=False).head(20)

,school_name,vacancies,total_application,popularity,match_on
113,Princess Elizabeth,200,392,1.960000,PRINCESSELIZABETH
100,Northland,240,413,1.720833,NORTHLAND
92,Nan Hua,240,391,1.629167,NANHUA
91,Nan Chiau,300,469,1.563333,NANCHIAU
66,Horizon,240,362,1.508333,HORIZON
31,CHIJ St. Nicholas Girls’,210,315,1.500000,CHIJSTNICHOLASGIRLS
141,St. Hilda’s,270,402,1.488889,STHILDAS
112,Poi Ching,240,357,1.487500,POICHING
58,Gongshang,210,310,1.476190,GONGSHANG
152,Temasek,200,290,1.450000,TEMASEK


There appears to be some nuances by phase but generally looks ok to use as a ranking - most searches of primary school ranking only consider certain [phases](https://sgschooling.com/phases) of the application process. 

## Add a percentile rank column to group schools

In [4]:
primary_ranks_cleaned['percentile_rank'] = primary_ranks_cleaned['popularity'].rank(pct=True)
primary_ranks_cleaned

,school_name,vacancies,total_application,popularity,match_on,percentile_rank
0,Admiralty,210,264,1.257143,ADMIRALTY,0.878453
1,Ahmad Ibrahim,210,84,0.400000,AHMADIBRAHIM,0.022099
2,Ai Tong,300,364,1.213333,AITONG,0.834254
3,Alexandra,200,223,1.115000,ALEXANDRA,0.596685
4,Anchor Green,240,271,1.129167,ANCHORGREEN,0.649171
...,...,...,...,...,...,...
176,Zhangde,210,223,1.061905,ZHANGDE,0.480663
177,Zhenghua,240,253,1.054167,ZHENGHUA,0.439227
178,Zhonghua,180,83,0.461111,ZHONGHUA,0.049724
179,Valour,280,295,1.053571,VALOUR,0.430939


In [5]:
schools = 'data/school-directory-and-information/general-information-of-schools.csv'

all_schools = pd.read_csv(schools)

primary = all_schools[all_schools['mainlevel_code']=='PRIMARY']
primary


,school_name,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,nature_code,session_code,mainlevel_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code
0,ADMIRALTY PRIMARY SCHOOL,https://admiraltypri.moe.edu.sg/,11 WOODLANDS CIRCLE,738907,63620598,na,63627512,na,ADMIRALTY_PS@MOE.EDU.SG,Admiralty Station,...,CO-ED SCHOOL,FULL DAY,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
2,AHMAD IBRAHIM PRIMARY SCHOOL,http://www.ahmadibrahimpri.moe.edu.sg,10 YISHUN STREET 11,768643,67592906,na,67592927,na,aips@moe.edu.sg,Yishun,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
4,AI TONG SCHOOL,http://www.aitong.moe.edu.sg,100 Bright Hill Drive,579646,64547672,na,64532726,na,AITONG_SCH@MOE.EDU.SG,Bishan MRT,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,Yes,No,No,No,Chinese,na,na
5,ALEXANDRA PRIMARY SCHOOL,http://alexandrapri.moe.edu.sg,2A Prince Charles Crescent,159016,62485400,na,62485409,na,alexandra_ps@moe.edu.sg,Redhill Station Tiong Bahru Station,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
6,ANCHOR GREEN PRIMARY SCHOOL,http://www.anchorgreenpri.moe.edu.sg,31 Anchorvale Drive,544969,68861356,na,63159825,na,anchorgreen_ps@moe.edu.sg,MRT : NE16-Sengkang LRT : SW7-TongKang,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,YUHUA PRIMARY SCHOOL,http://www.yuhuapri.moe.edu.sg,158 JURONG EAST STREET 24,609558,65605062,na,65648278,na,YUHUA_PS@MOE.EDU.SG,"Jurong East, Bukit Batok",...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
338,YUMIN PRIMARY SCHOOL,http://www.yuminpri.moe.edu.sg,3 TAMPINES STREET 21,529393,67811262,na,67845554,na,YUMIN_PS@MOE.EDU.SG,Tampines MRT Station,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,na
341,ZHANGDE PRIMARY SCHOOL,http://www.zhangdepri.moe.edu.sg/,51 Jalan Membina,169485,62740357,na,62763042,na,ZHANGDE_PS@MOE.EDU.SG,Tiong Bahru MRT Station,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
342,ZHENGHUA PRIMARY SCHOOL,http://www.zhenghuapri.moe.edu.sg,9 Fajar Road,679002,67697478,na,67636459,na,zhenghua_ps@moe.edu.sg,LRT: Fajar Station,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil


In [6]:
# clean school name to match strings
primary['match_on'] = primary['school_name'].str.replace('PRIMARY','')
primary['match_on'] = primary['match_on'].str.replace('SCHOOL','')
primary['match_on'] = primary['match_on'].str.replace("'",'')
primary['match_on'] = primary['match_on'].str.replace(".",'')
primary['match_on'] = primary['match_on'].str.replace(' ','')
primary['match_on'] = primary['match_on'].str.replace('(','')
primary['match_on'] = primary['match_on'].str.replace(')','')
primary['match_on'] = primary['match_on'].str.strip()
primary

/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

,school_name,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,session_code,mainlevel_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code,match_on
0,ADMIRALTY PRIMARY SCHOOL,https://admiraltypri.moe.edu.sg/,11 WOODLANDS CIRCLE,738907,63620598,na,63627512,na,ADMIRALTY_PS@MOE.EDU.SG,Admiralty Station,...,FULL DAY,PRIMARY,No,No,No,No,Chinese,Malay,Tamil,ADMIRALTY
2,AHMAD IBRAHIM PRIMARY SCHOOL,http://www.ahmadibrahimpri.moe.edu.sg,10 YISHUN STREET 11,768643,67592906,na,67592927,na,aips@moe.edu.sg,Yishun,...,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil,AHMADIBRAHIM
4,AI TONG SCHOOL,http://www.aitong.moe.edu.sg,100 Bright Hill Drive,579646,64547672,na,64532726,na,AITONG_SCH@MOE.EDU.SG,Bishan MRT,...,SINGLE SESSION,PRIMARY,Yes,No,No,No,Chinese,na,na,AITONG
5,ALEXANDRA PRIMARY SCHOOL,http://alexandrapri.moe.edu.sg,2A Prince Charles Crescent,159016,62485400,na,62485409,na,alexandra_ps@moe.edu.sg,Redhill Station Tiong Bahru Station,...,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil,ALEXANDRA
6,ANCHOR GREEN PRIMARY SCHOOL,http://www.anchorgreenpri.moe.edu.sg,31 Anchorvale Drive,544969,68861356,na,63159825,na,anchorgreen_ps@moe.edu.sg,MRT : NE16-Sengkang LRT : SW7-TongKang,...,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil,ANCHORGREEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,YUHUA PRIMARY SCHOOL,http://www.yuhuapri.moe.edu.sg,158 JURONG EAST STREET 24,609558,65605062,na,65648278,na,YUHUA_PS@MOE.EDU.SG,"Jurong East, Bukit Batok",...,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil,YUHUA
338,YUMIN PRIMARY SCHOOL,http://www.yuminpri.moe.edu.sg,3 TAMPINES STREET 21,529393,67811262,na,67845554,na,YUMIN_PS@MOE.EDU.SG,Tampines MRT Station,...,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,na,YUMIN
341,ZHANGDE PRIMARY SCHOOL,http://www.zhangdepri.moe.edu.sg/,51 Jalan Membina,169485,62740357,na,62763042,na,ZHANGDE_PS@MOE.EDU.SG,Tiong Bahru MRT Station,...,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil,ZHANGDE
342,ZHENGHUA PRIMARY SCHOOL,http://www.zhenghuapri.moe.edu.sg,9 Fajar Road,679002,67697478,na,67636459,na,zhenghua_ps@moe.edu.sg,LRT: Fajar Station,...,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil,ZHENGHUA


In [7]:
# merge the 2 dataframes
primary_ranked = primary.merge(primary_ranks_cleaned, left_on='match_on',right_on='match_on',how='left',indicator=True)
primary_ranked

,school_name_x,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,mothertongue1_code,mothertongue2_code,mothertongue3_code,match_on,school_name_y,vacancies,total_application,popularity,percentile_rank,_merge
0,ADMIRALTY PRIMARY SCHOOL,https://admiraltypri.moe.edu.sg/,11 WOODLANDS CIRCLE,738907,63620598,na,63627512,na,ADMIRALTY_PS@MOE.EDU.SG,Admiralty Station,...,Chinese,Malay,Tamil,ADMIRALTY,Admiralty,210,264.0,1.257143,0.878453,both
1,AHMAD IBRAHIM PRIMARY SCHOOL,http://www.ahmadibrahimpri.moe.edu.sg,10 YISHUN STREET 11,768643,67592906,na,67592927,na,aips@moe.edu.sg,Yishun,...,Chinese,Malay,Tamil,AHMADIBRAHIM,Ahmad Ibrahim,210,84.0,0.400000,0.022099,both
2,AI TONG SCHOOL,http://www.aitong.moe.edu.sg,100 Bright Hill Drive,579646,64547672,na,64532726,na,AITONG_SCH@MOE.EDU.SG,Bishan MRT,...,Chinese,na,na,AITONG,Ai Tong,300,364.0,1.213333,0.834254,both
3,ALEXANDRA PRIMARY SCHOOL,http://alexandrapri.moe.edu.sg,2A Prince Charles Crescent,159016,62485400,na,62485409,na,alexandra_ps@moe.edu.sg,Redhill Station Tiong Bahru Station,...,Chinese,Malay,Tamil,ALEXANDRA,Alexandra,200,223.0,1.115000,0.596685,both
4,ANCHOR GREEN PRIMARY SCHOOL,http://www.anchorgreenpri.moe.edu.sg,31 Anchorvale Drive,544969,68861356,na,63159825,na,anchorgreen_ps@moe.edu.sg,MRT : NE16-Sengkang LRT : SW7-TongKang,...,Chinese,Malay,Tamil,ANCHORGREEN,Anchor Green,240,271.0,1.129167,0.649171,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,YUHUA PRIMARY SCHOOL,http://www.yuhuapri.moe.edu.sg,158 JURONG EAST STREET 24,609558,65605062,na,65648278,na,YUHUA_PS@MOE.EDU.SG,"Jurong East, Bukit Batok",...,Chinese,Malay,Tamil,YUHUA,Yuhua,240,165.0,0.687500,0.149171,both
179,YUMIN PRIMARY SCHOOL,http://www.yuminpri.moe.edu.sg,3 TAMPINES STREET 21,529393,67811262,na,67845554,na,YUMIN_PS@MOE.EDU.SG,Tampines MRT Station,...,Chinese,Malay,na,YUMIN,Yumin,180,191.0,1.061111,0.469613,both
180,ZHANGDE PRIMARY SCHOOL,http://www.zhangdepri.moe.edu.sg/,51 Jalan Membina,169485,62740357,na,62763042,na,ZHANGDE_PS@MOE.EDU.SG,Tiong Bahru MRT Station,...,Chinese,Malay,Tamil,ZHANGDE,Zhangde,210,223.0,1.061905,0.480663,both
181,ZHENGHUA PRIMARY SCHOOL,http://www.zhenghuapri.moe.edu.sg,9 Fajar Road,679002,67697478,na,67636459,na,zhenghua_ps@moe.edu.sg,LRT: Fajar Station,...,Chinese,Malay,Tamil,ZHENGHUA,Zhenghua,240,253.0,1.054167,0.439227,both


In [8]:
primary_ranked[primary_ranked['_merge']=='left_only']

,school_name_x,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,mothertongue1_code,mothertongue2_code,mothertongue3_code,match_on,school_name_y,vacancies,total_application,popularity,percentile_rank,_merge
44,EUNOS PRIMARY SCHOOL,http://www.eunospri.moe.edu.sg/,95 JALAN EUNOS,419529,67463336,na,67488554,na,eunos_ps@moe.edu.sg,Nearest MRT Station - Eunos,...,Chinese,Malay,Tamil,EUNOS,NaN,NaN,NaN,NaN,NaN,left_only
61,GUANGYANG PRIMARY SCHOOL,http://www.guangyangpri.moe.edu.sg,6 Bishan Street 12,579806,62580455,na,62580010,na,GYPS@MOE.EDU.SG,Bishan MRT,...,Chinese,Malay,Tamil,GUANGYANG,NaN,NaN,NaN,NaN,NaN,left_only
75,JUYING PRIMARY SCHOOL,http://www.juyingpri.moe.edu.sg/,31 JURONG WEST STREET 91,649037,67932039,na,67932040,na,JUYING_PS@MOE.EDU.SG,Boon Lay MRT Station (Jurong Point),...,Chinese,Malay,Tamil,JUYING,NaN,NaN,NaN,NaN,NaN,left_only
113,PIONEER PRIMARY SCHOOL,http://www.pioneerpri.moe.edu.sg,23 JURONG WEST STREET 81,649076,67932397,na,67933974,na,pioneer_ps@moe.edu.sg,Boon Lay Station,...,Chinese,Malay,Tamil,PIONEER,NaN,NaN,NaN,NaN,NaN,left_only
147,STAMFORD PRIMARY SCHOOL,http://www.stamfordpri.moe.edu.sg,1 VICTORIA LANE,198423,62945074,na,62919262,na,STAMFORD_PS@MOE.EDU.SG,"Bugis MRT Station, Jalan Besar MRT Station",...,Chinese,Malay,Tamil,STAMFORD,NaN,NaN,NaN,NaN,NaN,left_only


In [9]:
primary_ranked[['vacancies','total_application','popularity','percentile_rank']]= primary_ranked[['vacancies','total_application','popularity','percentile_rank']].fillna(value=0)
primary_ranked.drop(columns=['_merge'],inplace=True)
primary_ranked['address'] = primary_ranked['address'].str.upper()
primary_ranked['match_on'] = primary_ranked['school_name_x']
primary_ranked['match_on'] = primary_ranked['match_on'].str.replace("'",'')
primary_ranked['match_on'] = primary_ranked['match_on'].str.replace(".",'')
primary_ranked['match_on'] = primary_ranked['match_on'].str.replace(' ','')
primary_ranked['match_on'] = primary_ranked['match_on'].str.replace('(','')
primary_ranked['match_on'] = primary_ranked['match_on'].str.replace(')','')
primary_ranked['match_on'] = primary_ranked['match_on'].str.strip()
primary_ranked

/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  
/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  
/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  if __name__ == '__main__':


,school_name_x,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code,match_on,school_name_y,vacancies,total_application,popularity,percentile_rank
0,ADMIRALTY PRIMARY SCHOOL,https://admiraltypri.moe.edu.sg/,11 WOODLANDS CIRCLE,738907,63620598,na,63627512,na,ADMIRALTY_PS@MOE.EDU.SG,Admiralty Station,...,No,Chinese,Malay,Tamil,ADMIRALTYPRIMARYSCHOOL,Admiralty,210,264.0,1.257143,0.878453
1,AHMAD IBRAHIM PRIMARY SCHOOL,http://www.ahmadibrahimpri.moe.edu.sg,10 YISHUN STREET 11,768643,67592906,na,67592927,na,aips@moe.edu.sg,Yishun,...,No,Chinese,Malay,Tamil,AHMADIBRAHIMPRIMARYSCHOOL,Ahmad Ibrahim,210,84.0,0.400000,0.022099
2,AI TONG SCHOOL,http://www.aitong.moe.edu.sg,100 BRIGHT HILL DRIVE,579646,64547672,na,64532726,na,AITONG_SCH@MOE.EDU.SG,Bishan MRT,...,No,Chinese,na,na,AITONGSCHOOL,Ai Tong,300,364.0,1.213333,0.834254
3,ALEXANDRA PRIMARY SCHOOL,http://alexandrapri.moe.edu.sg,2A PRINCE CHARLES CRESCENT,159016,62485400,na,62485409,na,alexandra_ps@moe.edu.sg,Redhill Station Tiong Bahru Station,...,No,Chinese,Malay,Tamil,ALEXANDRAPRIMARYSCHOOL,Alexandra,200,223.0,1.115000,0.596685
4,ANCHOR GREEN PRIMARY SCHOOL,http://www.anchorgreenpri.moe.edu.sg,31 ANCHORVALE DRIVE,544969,68861356,na,63159825,na,anchorgreen_ps@moe.edu.sg,MRT : NE16-Sengkang LRT : SW7-TongKang,...,No,Chinese,Malay,Tamil,ANCHORGREENPRIMARYSCHOOL,Anchor Green,240,271.0,1.129167,0.649171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,YUHUA PRIMARY SCHOOL,http://www.yuhuapri.moe.edu.sg,158 JURONG EAST STREET 24,609558,65605062,na,65648278,na,YUHUA_PS@MOE.EDU.SG,"Jurong East, Bukit Batok",...,No,Chinese,Malay,Tamil,YUHUAPRIMARYSCHOOL,Yuhua,240,165.0,0.687500,0.149171
179,YUMIN PRIMARY SCHOOL,http://www.yuminpri.moe.edu.sg,3 TAMPINES STREET 21,529393,67811262,na,67845554,na,YUMIN_PS@MOE.EDU.SG,Tampines MRT Station,...,No,Chinese,Malay,na,YUMINPRIMARYSCHOOL,Yumin,180,191.0,1.061111,0.469613
180,ZHANGDE PRIMARY SCHOOL,http://www.zhangdepri.moe.edu.sg/,51 JALAN MEMBINA,169485,62740357,na,62763042,na,ZHANGDE_PS@MOE.EDU.SG,Tiong Bahru MRT Station,...,No,Chinese,Malay,Tamil,ZHANGDEPRIMARYSCHOOL,Zhangde,210,223.0,1.061905,0.480663
181,ZHENGHUA PRIMARY SCHOOL,http://www.zhenghuapri.moe.edu.sg,9 FAJAR ROAD,679002,67697478,na,67636459,na,zhenghua_ps@moe.edu.sg,LRT: Fajar Station,...,No,Chinese,Malay,Tamil,ZHENGHUAPRIMARYSCHOOL,Zhenghua,240,253.0,1.054167,0.439227


In [46]:
engine = create_engine(f'postgresql+psycopg2://{Capstone_AWS_PG.username}:{Capstone_AWS_PG.password}@{Capstone_AWS_PG.host}/capstone', echo=False)

sql_query = 'select * from sg_buildings_postal;'
sql_alc_cnxn = engine.connect()

postal = pd.read_sql(sql_query, sql_alc_cnxn)
postal['sch_match'] = postal['searchval']
postal['sch_match'] = postal['sch_match'].str.replace('SAINT','ST')
postal['sch_match'] = postal['sch_match'].str.replace("(U/C)",'')
postal['sch_match'] = postal['sch_match'].str.replace("'",'')
postal['sch_match'] = postal['sch_match'].str.replace(".",'')
postal['sch_match'] = postal['sch_match'].str.replace(' ','')
postal['sch_match'] = postal['sch_match'].str.replace('(','')
postal['sch_match'] = postal['sch_match'].str.replace(')','')
postal['sch_match'] = postal['sch_match'].str.strip()
postal['sch_match'] = postal['sch_match'].str.replace("METHODISTGIRLSSCHOOL","METHODISTGIRLSSCHOOLPRIMARY")
postal['sch_match'] = postal['sch_match'].str.replace("PAYALEBARMETHODISTGIRLSSCHOOLPRIMARYPRIMARY","PAYALEBARMETHODISTGIRLSSCHOOLPRIMARY")
postal['sch_match'] = postal['sch_match'].str.replace("SINGAPORECHINESEGIRLSSCHOOLPRIMARY","SINGAPORECHINESEGIRLSPRIMARYSCHOOL")

postal

/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == '__main__':
/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  # This is added back by InteractiveShellApp.init_path()
/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  del sys.path[0]
/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: The default value of regex will change from

,address,blk_no,building,latitude,longitude,longtitude,postal,road_name,searchval,x,y,sch_match
0,101A BAYFRONT AVENUE TEMPORARY SITE OFFICE SIN...,101A,TEMPORARY SITE OFFICE,1.275697,103.855652,103.855652,18895,BAYFRONT AVENUE,TEMPORARY SITE OFFICE,30485.511362,28685.612665,TEMPORARYSITEOFFICE
1,1 STRAITS BOULEVARD SINGAPORE CHINESE CULTURAL...,1,SINGAPORE CHINESE CULTURAL CENTRE,1.275829,103.849576,103.849576,18906,STRAITS BOULEVARD,SINGAPORE CHINESE CULTURAL CENTRE,29809.365407,28700.236127,SINGAPORECHINESECULTURALCENTRE
2,11A STRAITS BOULEVARD TEMPORARY SITE OFFICE SI...,11A,TEMPORARY SITE OFFICE,1.274950,103.851665,103.851665,18907,STRAITS BOULEVARD,TEMPORARY SITE OFFICE,30041.838898,28602.987244,TEMPORARYSITEOFFICE
3,2 CENTRAL BOULEVARD IOI CENTRAL BOULEVARD TOWE...,2,IOI CENTRAL BOULEVARD TOWERS,1.279777,103.851513,103.851513,18916,CENTRAL BOULEVARD,IOI CENTRAL BOULEVARD TOWERS,30024.919852,29136.807026,IOICENTRALBOULEVARDTOWERS
4,21 PARK STREET DBS MARINA BAY MRT STATION SING...,21,DBS MARINA BAY MRT STATION,1.276427,103.854598,103.854598,18925,PARK STREET,DBS MARINA BAY MRT STATION,30368.205612,28766.381902,DBSMARINABAYMRTSTATION
...,...,...,...,...,...,...,...,...,...,...,...,...
142881,100A KRANJI LOOP SINGAPORE 887327,100A,NIL,1.433629,103.758648,103.758648,887327,KRANJI LOOP,100A KRANJI LOOP SINGAPORE 887327,19690.280996,46149.118386,100AKRANJILOOPSINGAPORE887327
142882,A PASIR PANJANG ROAD SINGAPORE 887328,A,NIL,1.277170,103.795840,103.795840,887328,PASIR PANJANG ROAD,A PASIR PANJANG ROAD SINGAPORE 887328,23828.902741,28848.553424,APASIRPANJANGROADSINGAPORE887328
142883,PULAU BUKOM SINGAPORE 903808,,NIL,1.235596,103.768645,103.768645,903808,PULAU BUKOM,PULAU BUKOM SINGAPORE 903808,20802.263821,24251.557450,PULAUBUKOMSINGAPORE903808
142884,GATE C7 AIRPORT CARGO ROAD CHANGI ANIMAL AND P...,GATE C7,CHANGI ANIMAL AND PLANT QUARANTINE CENTRE,1.375285,103.996737,103.996737,918104,AIRPORT CARGO ROAD,CHANGI ANIMAL AND PLANT QUARANTINE CENTRE,46186.488419,39698.194002,CHANGIANIMALANDPLANTQUARANTINECENTRE


In [42]:
postal[postal['searchval'].str.contains('METHODIST GIRL')]

,address,blk_no,building,latitude,longitude,longtitude,postal,road_name,searchval,x,y,sch_match
84081,298 LORONG AH SOO COMMIT LEARNING SCHOOLHOUSE ...,298,COMMIT LEARNING SCHOOLHOUSE @ PAYA LEBAR METHO...,1.350255,103.884847,103.884847,536741,LORONG AH SOO,COMMIT LEARNING SCHOOLHOUSE @ PAYA LEBAR METHO...,33734.522469,36929.865687,COMMITLEARNINGSCHOOLHOUSE@PAYALEBARMETHODISTGI...
84082,298 LORONG AH SOO PAYA LEBAR METHODIST GIRLS' ...,298,PAYA LEBAR METHODIST GIRLS' SCHOOL (PRIMARY),1.349473,103.884971,103.884971,536741,LORONG AH SOO,PAYA LEBAR METHODIST GIRLS' SCHOOL (PRIMARY),33748.376095,36843.418782,PAYALEBARMETHODISTGIRLSSCHOOLPRIMARYPRIMARY
84083,296 LORONG AH SOO PAYA LEBAR METHODIST GIRLS' ...,296,PAYA LEBAR METHODIST GIRLS' SCHOOL (SECONDARY),1.350968,103.885120,103.885120,536742,LORONG AH SOO,PAYA LEBAR METHODIST GIRLS' SCHOOL (SECONDARY),33764.922058,37008.706157,PAYALEBARMETHODISTGIRLSSCHOOLPRIMARYSECONDARY
109702,11 BLACKMORE DRIVE COMMIT LEARNING SCHOOHHOUSE...,11,COMMIT LEARNING SCHOOHHOUSE @ METHODIST GIRLS'...,1.332662,103.783382,103.783382,599986,BLACKMORE DRIVE,COMMIT LEARNING SCHOOHHOUSE @ METHODIST GIRLS'...,22442.590980,34984.581056,COMMITLEARNINGSCHOOHHOUSE@METHODISTGIRLSSCHOOL...
109704,11 BLACKMORE DRIVE METHODIST GIRLS' SCHOOL SIN...,11,METHODIST GIRLS' SCHOOL,1.332948,103.783393,103.783393,599986,BLACKMORE DRIVE,METHODIST GIRLS' SCHOOL,22443.804211,35016.191455,METHODISTGIRLSSCHOOLPRIMARY


In [43]:
primary_ranked[primary_ranked['school_name_x'].str.contains('METHODIST GIRL')]

,school_name_x,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code,match_on,school_name_y,vacancies,total_application,popularity,percentile_rank
89,METHODIST GIRLS' SCHOOL (PRIMARY),http://www.mgs.sch.edu.sg,11 BLACKMORE DRIVE,599986,64694800,na,64624166,na,mgps@moe.edu.sg,King Albert Park MRT,...,No,Chinese,Malay,Tamil,METHODISTGIRLSSCHOOLPRIMARY,Methodist Girls’,210,252.0,1.200000,0.803867
108,PAYA LEBAR METHODIST GIRLS' SCHOOL (PRIMARY),http://www.payalebarmethodistgirlspri.moe.edu.sg/,298 LORONG AH SOO,536741,62862795,na,62803915,na,PLMGPS@MOE.EDU.SG,"Kovan (NE13), Serangoon(NE12/CC13)",...,No,Chinese,Malay,Tamil,PAYALEBARMETHODISTGIRLSSCHOOLPRIMARY,Paya Lebar Methodist Girls’,270,314.0,1.162963,0.734807


In [47]:
primary_ranked_location = primary_ranked.merge(postal, left_on='match_on',right_on='sch_match',how='left',indicator=True)
primary_ranked_location

,school_name_x,url_address,address_x,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,latitude,longitude,longtitude,postal,road_name,searchval,x,y,sch_match,_merge
0,ADMIRALTY PRIMARY SCHOOL,https://admiraltypri.moe.edu.sg/,11 WOODLANDS CIRCLE,738907,63620598,na,63627512,na,ADMIRALTY_PS@MOE.EDU.SG,Admiralty Station,...,1.442635,103.800040,103.800040,738907,WOODLANDS CIRCLE,ADMIRALTY PRIMARY SCHOOL,24296.625086,47144.770059,ADMIRALTYPRIMARYSCHOOL,both
1,AHMAD IBRAHIM PRIMARY SCHOOL,http://www.ahmadibrahimpri.moe.edu.sg,10 YISHUN STREET 11,768643,67592906,na,67592927,na,aips@moe.edu.sg,Yishun,...,1.433153,103.832942,103.832942,768643,YISHUN STREET 11,AHMAD IBRAHIM PRIMARY SCHOOL,27958.137143,46096.262768,AHMADIBRAHIMPRIMARYSCHOOL,both
2,AI TONG SCHOOL,http://www.aitong.moe.edu.sg,100 BRIGHT HILL DRIVE,579646,64547672,na,64532726,na,AITONG_SCH@MOE.EDU.SG,Bishan MRT,...,1.360583,103.833020,103.833020,579646,BRIGHT HILL DRIVE,AI TONG SCHOOL,27966.808830,38071.919118,AITONGSCHOOL,both
3,ALEXANDRA PRIMARY SCHOOL,http://alexandrapri.moe.edu.sg,2A PRINCE CHARLES CRESCENT,159016,62485400,na,62485409,na,alexandra_ps@moe.edu.sg,Redhill Station Tiong Bahru Station,...,1.291334,103.824425,103.824425,159016,PRINCE CHARLES CRESCENT,ALEXANDRA PRIMARY SCHOOL,27010.185482,30414.716970,ALEXANDRAPRIMARYSCHOOL,both
4,ANCHOR GREEN PRIMARY SCHOOL,http://www.anchorgreenpri.moe.edu.sg,31 ANCHORVALE DRIVE,544969,68861356,na,63159825,na,anchorgreen_ps@moe.edu.sg,MRT : NE16-Sengkang LRT : SW7-TongKang,...,1.390370,103.887165,103.887165,544969,ANCHORVALE DRIVE,ANCHOR GREEN PRIMARY SCHOOL,33992.445947,41365.632950,ANCHORGREENPRIMARYSCHOOL,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,YUHUA PRIMARY SCHOOL,http://www.yuhuapri.moe.edu.sg,158 JURONG EAST STREET 24,609558,65605062,na,65648278,na,YUHUA_PS@MOE.EDU.SG,"Jurong East, Bukit Batok",...,1.342802,103.741106,103.741106,609558,JURONG EAST STREET 24,YUHUA PRIMARY SCHOOL,17737.713011,36105.966237,YUHUAPRIMARYSCHOOL,both
187,YUMIN PRIMARY SCHOOL,http://www.yuminpri.moe.edu.sg,3 TAMPINES STREET 21,529393,67811262,na,67845554,na,YUMIN_PS@MOE.EDU.SG,Tampines MRT Station,...,1.351292,103.950462,103.950462,529393,TAMPINES STREET 21,YUMIN PRIMARY SCHOOL,41036.744732,37044.809263,YUMINPRIMARYSCHOOL,both
188,ZHANGDE PRIMARY SCHOOL,http://www.zhangdepri.moe.edu.sg/,51 JALAN MEMBINA,169485,62740357,na,62763042,na,ZHANGDE_PS@MOE.EDU.SG,Tiong Bahru MRT Station,...,1.284212,103.825952,103.825952,169485,JALAN MEMBINA,ZHANGDE PRIMARY SCHOOL,27180.146899,29627.107530,ZHANGDEPRIMARYSCHOOL,both
189,ZHENGHUA PRIMARY SCHOOL,http://www.zhenghuapri.moe.edu.sg,9 FAJAR ROAD,679002,67697478,na,67636459,na,zhenghua_ps@moe.edu.sg,LRT: Fajar Station,...,1.379549,103.769314,103.769314,679002,FAJAR ROAD,ZHENGHUA PRIMARY SCHOOL,20877.039543,40169.116913,ZHENGHUAPRIMARYSCHOOL,both


In [48]:
primary_ranked_location[primary_ranked_location['_merge']=='left_only']

,school_name_x,url_address,address_x,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,latitude,longitude,longtitude,postal,road_name,searchval,x,y,sch_match,_merge


In [49]:
cols_to_keep = ['school_name_x',
                'url_address',
                'dgp_code',
                'zone_code',
                'type_code',
                'nature_code',
                'session_code',
                'mainlevel_code',
                'sap_ind',
                'autonomous_ind',
                'gifted_ind',
                'ip_ind',
                'mothertongue1_code',
                'mothertongue2_code',
                'mothertongue3_code',
                'vacancies',
                'total_application',
                'popularity',
                'percentile_rank',
                'address_y',
                'blk_no',
                'building',
                'latitude',
                'longitude',
                'longtitude',
                'postal',
                'road_name',
                'searchval',
                'x',
                'y',]

primary_ranked_final = primary_ranked_location[cols_to_keep]
primary_ranked_final

,school_name_x,url_address,dgp_code,zone_code,type_code,nature_code,session_code,mainlevel_code,sap_ind,autonomous_ind,...,blk_no,building,latitude,longitude,longtitude,postal,road_name,searchval,x,y
0,ADMIRALTY PRIMARY SCHOOL,https://admiraltypri.moe.edu.sg/,WOODLANDS,NORTH,GOVERNMENT SCHOOL,CO-ED SCHOOL,FULL DAY,PRIMARY,No,No,...,11,ADMIRALTY PRIMARY SCHOOL,1.442635,103.800040,103.800040,738907,WOODLANDS CIRCLE,ADMIRALTY PRIMARY SCHOOL,24296.625086,47144.770059
1,AHMAD IBRAHIM PRIMARY SCHOOL,http://www.ahmadibrahimpri.moe.edu.sg,YISHUN,NORTH,GOVERNMENT SCHOOL,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,...,10,AHMAD IBRAHIM PRIMARY SCHOOL,1.433153,103.832942,103.832942,768643,YISHUN STREET 11,AHMAD IBRAHIM PRIMARY SCHOOL,27958.137143,46096.262768
2,AI TONG SCHOOL,http://www.aitong.moe.edu.sg,BISHAN,SOUTH,GOVERNMENT-AIDED SCH,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,Yes,No,...,100,AI TONG SCHOOL,1.360583,103.833020,103.833020,579646,BRIGHT HILL DRIVE,AI TONG SCHOOL,27966.808830,38071.919118
3,ALEXANDRA PRIMARY SCHOOL,http://alexandrapri.moe.edu.sg,BUKIT MERAH,SOUTH,GOVERNMENT SCHOOL,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,...,2A,ALEXANDRA PRIMARY SCHOOL,1.291334,103.824425,103.824425,159016,PRINCE CHARLES CRESCENT,ALEXANDRA PRIMARY SCHOOL,27010.185482,30414.716970
4,ANCHOR GREEN PRIMARY SCHOOL,http://www.anchorgreenpri.moe.edu.sg,SENG KANG,NORTH,GOVERNMENT SCHOOL,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,...,31,ANCHOR GREEN PRIMARY SCHOOL,1.390370,103.887165,103.887165,544969,ANCHORVALE DRIVE,ANCHOR GREEN PRIMARY SCHOOL,33992.445947,41365.632950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,YUHUA PRIMARY SCHOOL,http://www.yuhuapri.moe.edu.sg,JURONG EAST,WEST,GOVERNMENT SCHOOL,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,...,158,YUHUA PRIMARY SCHOOL,1.342802,103.741106,103.741106,609558,JURONG EAST STREET 24,YUHUA PRIMARY SCHOOL,17737.713011,36105.966237
187,YUMIN PRIMARY SCHOOL,http://www.yuminpri.moe.edu.sg,TAMPINES,EAST,GOVERNMENT SCHOOL,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,...,3,YUMIN PRIMARY SCHOOL,1.351292,103.950462,103.950462,529393,TAMPINES STREET 21,YUMIN PRIMARY SCHOOL,41036.744732,37044.809263
188,ZHANGDE PRIMARY SCHOOL,http://www.zhangdepri.moe.edu.sg/,BUKIT MERAH,SOUTH,GOVERNMENT SCHOOL,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,...,51,ZHANGDE PRIMARY SCHOOL,1.284212,103.825952,103.825952,169485,JALAN MEMBINA,ZHANGDE PRIMARY SCHOOL,27180.146899,29627.107530
189,ZHENGHUA PRIMARY SCHOOL,http://www.zhenghuapri.moe.edu.sg,BUKIT PANJANG,WEST,GOVERNMENT SCHOOL,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,...,9,ZHENGHUA PRIMARY SCHOOL,1.379549,103.769314,103.769314,679002,FAJAR ROAD,ZHENGHUA PRIMARY SCHOOL,20877.039543,40169.116913


In [55]:
primary_ranked_final[primary_ranked_final.duplicated(subset=['school_name_x'])]

,school_name_x,url_address,dgp_code,zone_code,type_code,nature_code,session_code,mainlevel_code,sap_ind,autonomous_ind,...,blk_no,building,latitude,longitude,longtitude,postal,road_name,searchval,x,y
10,ANGSANA PRIMARY SCHOOL,www.angsanapri.moe.edu.sg,TAMPINES,EAST,GOVERNMENT SCHOOL,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,...,3,ANGSANA PRIMARY SCHOOL,1.348553,103.951808,103.951808,529366,TAMPINES STREET 22,ANGSANA PRIMARY SCHOOL,41186.574308,36741.962195
64,HAIG GIRLS' SCHOOL,http://www.haiggirls.moe.edu.sg,GEYLANG,EAST,GOVERNMENT SCHOOL,GIRLS' SCHOOL,SINGLE SESSION,PRIMARY,No,No,...,51,HAIG GIRLS' SCHOOL (U/C),1.311539,103.903394,103.903394,427072,KOON SENG ROAD,HAIG GIRLS' SCHOOL (U/C),35798.686756,32648.958479
83,KUO CHUAN PRESBYTERIAN PRIMARY SCHOOL,http://www.kuochuanpresbyterianpri.moe.edu.sg,BISHAN,SOUTH,GOVERNMENT-AIDED SCH,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,...,8,KUO CHUAN PRESBYTERIAN PRIMARY SCHOOL (U/C),1.349310,103.854300,103.854300,579793,BISHAN STREET 13,KUO CHUAN PRESBYTERIAN PRIMARY SCHOOL (U/C),30335.028936,36825.386086
84,KUO CHUAN PRESBYTERIAN PRIMARY SCHOOL,http://www.kuochuanpresbyterianpri.moe.edu.sg,BISHAN,SOUTH,GOVERNMENT-AIDED SCH,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,...,8B,KUO CHUAN PRESBYTERIAN PRIMARY SCHOOL,1.349783,103.854529,103.854529,579802,BISHAN STREET 13,KUO CHUAN PRESBYTERIAN PRIMARY SCHOOL,30360.509977,36877.680687
88,MAHA BODHI SCHOOL,http://www.mahabodhi.moe.edu.sg/,GEYLANG,EAST,GOVERNMENT-AIDED SCH,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,Yes,No,...,12,MAHA BODHI SCHOOL,1.328471,103.901299,103.901299,408932,UBI AVENUE 1,MAHA BODHI SCHOOL,35565.497911,34521.179587
99,NANYANG PRIMARY SCHOOL,http://www.nyps.moe.edu.sg,BUKIT TIMAH,WEST,GOVERNMENT-AIDED SCH,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,Yes,No,...,51,NANYANG PRIMARY SCHOOL,1.321115,103.806468,103.806468,269460,CORONATION ROAD,NANYANG PRIMARY SCHOOL,25011.802968,33707.771958
150,ST. HILDA'S PRIMARY SCHOOL,http://www.shps.moe.edu.sg,TAMPINES,EAST,GOVERNMENT-AIDED SCH,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,...,2,SAINT HILDA'S PRIMARY SCHOOL (U/C),1.349350,103.937921,103.937921,529706,TAMPINES AVENUE 3,SAINT HILDA'S PRIMARY SCHOOL (U/C),39641.033036,36830.027257
153,ST. MARGARET'S PRIMARY SCHOOL,na,CENTRAL,SOUTH,GOVERNMENT-AIDED SCH,GIRLS' SCHOOL,SINGLE SESSION,PRIMARY,No,No,...,2,SAINT MARGARET'S PRIMARY SCHOOL,1.324392,103.881625,103.881625,387724,MATTAR ROAD,SAINT MARGARET'S PRIMARY SCHOOL,33376.001804,34070.052846


In [61]:
# drop schools under construction 
primary_ranked_final=primary_ranked_final[~primary_ranked_final['searchval'].str.contains("U/C")]

In [62]:
primary_ranked_final.to_sql('primary_schools_ranked_with_loc',sql_alc_cnxn, if_exists='replace', index=False)